# Instalacion dependencias

In [ ]:
# --- 1. INSTALACIÓN DE LIBRERÍAS ---
# Instalamos kagglehub y aseguramos librosa
!pip install -q kagglehub librosa numpy pandas datasets tqdm openpyxl soundfile

# Dataset SSI Entrenamiento

In [ ]:
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import os

def extract_features(audio_data, sample_rate, n_mfcc=40):
    """
    Extrae un conjunto de características de una señal de audio.
    Usamos la media de los MFCCs, Chroma y Mel Spectrogram.
    """
    features = {}

    # Asegurarse de que el audio sea flotante (requerido por librosa)
    if audio_data.dtype != np.float32:
        audio_data = audio_data.astype(np.float32)

    # MFCCs (Coeficientes Cepstrales en la Frecuencia Mel)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)

    for i, mfcc_val in enumerate(mfccs_mean):
        features[f'mfcc_{i+1}_mean'] = mfcc_val

    # Chroma (Características de Tono)
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    features['chroma_mean'] = np.mean(chroma)

    # Mel Spectrogram (Espectrograma en escala Mel)
    mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    features['mel_mean'] = np.mean(mel)

    # Contraste Espectral (Spectral Contrast)
    contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sample_rate)
    features['contrast_mean'] = np.mean(contrast)

    return features

# --- Script Principal ---

print("Iniciando el proceso...")

os.environ['TRANSFORMERS_TRUST_REMOTE_CODE'] = '1'
os.environ['HF_DATASETS_DISABLE_TORCHCODEC'] = '1'

dataset_name = "stapesai/ssi-speech-emotion-recognition"
SAMPLING_RATE = 16000

# 1. CARGAR DATOS - Sin Audio wrapper para evitar decodificación
print("Cargando el dataset...")
try:
    dataset = load_dataset(dataset_name, 'default', split='train')
except Exception as e:
    print(f"Error: {e}")
    dataset = load_dataset(dataset_name, split='train')

print(f"Dataset cargado. Total de muestras: {len(dataset)}")

# 2. EXTRAER CARACTERÍSTICAS
processed_data = []
print("Extrayendo características...")

# Acceder directamente a la tabla PyArrow para evitar decodificación
arrow_table = dataset.data

# Inspeccionar la estructura
print(f"Columnas disponibles: {arrow_table.column_names}")

# Procesar todas las muestras
for idx in tqdm(range(arrow_table.num_rows)):
    try:
        # Acceder directamente a la tabla PyArrow sin formateo
        row_dict = {
            name: arrow_table.column(name)[idx].as_py()
            for name in arrow_table.column_names
        }

        # Obtener info del audio y emoción
        file_path_info = row_dict.get('file_path')  # Dict con 'bytes' y 'path'
        emotion_label = row_dict.get('emotion')

        # Extraer audio bytes
        if isinstance(file_path_info, dict) and 'bytes' in file_path_info:
            audio_bytes = file_path_info['bytes']

            # Guardar temporalmente y cargar con librosa
            import tempfile
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
                tmp.write(audio_bytes)
                tmp_path = tmp.name
            try:
                audio_data, sample_rate = librosa.load(tmp_path, sr=SAMPLING_RATE)

                # Extraer características
                features = extract_features(audio_data, sample_rate)
                features['emotion'] = emotion_label
                processed_data.append(features)
            finally:
                os.remove(tmp_path)

    except Exception as e:
        pass  # Ignorar errores

print(f"\nSe procesaron {len(processed_data)} muestras")

# 3. CREAR Y GUARDAR EL CSV y XLSX
if processed_data:
    print("Creando DataFrame de Pandas...")
    df = pd.DataFrame(processed_data)

    # Poner 'emotion' primero
    cols = ['emotion'] + [col for col in df.columns if col != 'emotion']
    df = df[cols]

    # Guardar como CSV
    csv_filename = 'ssi_custom_features.csv'
    df.to_csv(csv_filename, index=False, sep=',', quoting=1)
    print(f"CSV guardado en: {csv_filename}")

    # Guardar como XLSX
    xlsx_filename = 'ssi_custom_features.xlsx'
    df.to_excel(xlsx_filename, index=False, sheet_name='Datos')

    print(f"\n¡Proceso completado!")
    print(f"XLSX guardado en: {xlsx_filename}")
    print(f"Total de registros: {len(df)}")
    print("\nVista previa de los datos:")
    print(df.head())
    print(f"\nForma del dataset: {df.shape}")
    print(f"Columnas: {list(df.columns[:10])}...")  # Mostrar solo primeras 10
else:
    print("No se procesaron datos.")

# Dataset RAVDESS Tests

In [ ]:

import kagglehub
import os
import glob
import librosa
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# --- 2. DESCARGA DEL DATASET ---
print("Descargando RAVDESS con kagglehub...")
# Descargamos la última versión del dataset
path_dataset = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

print(f"Dataset descargado en: {path_dataset}")

# --- 3. FUNCIÓN DE EXTRACCIÓN (Tu "matemática" original) ---
def extract_features(audio_data, sample_rate, n_mfcc=40):
    features = {}

    if audio_data.dtype != np.float32:
        audio_data = audio_data.astype(np.float32)

    # MFCCs
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    for i, mfcc_val in enumerate(mfccs_mean):
        features[f'mfcc_{i+1}_mean'] = mfcc_val

    # Chroma
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    features['chroma_mean'] = np.mean(chroma)

    # Mel Spectrogram
    mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    features['mel_mean'] = np.mean(mel)

    # Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sample_rate)
    features['contrast_mean'] = np.mean(contrast)

    return features

# --- 4. CONFIGURACIÓN ---
# Mapa de emociones (Asegúrate que coincidan con las de tu modelo en Orange)
emotion_map = {
    '01': 'NEU',
    '02': 'CAL',     # Si tu modelo no tiene CALM, coméntalo o cámbialo a NEUTRAL
    '03': 'HAP',
    '04': 'SAD',
    '05': 'ANG',
    '06': 'FEA',
    '07': 'DIS',
    '08': 'SUR'
}

processed_data = []
processed_filenames = set() # Para evitar duplicados
SAMPLE_RATE = 16000

print("\nIniciando procesamiento de audios...")

# --- 5. BUCLE PRINCIPAL ---
# Usamos os.walk para recorrer la carpeta descargada por kagglehub
for dirname, _, filenames in os.walk(path_dataset):
    for filename in filenames:
        if filename.endswith('.wav'):

            # FILTRO DE DUPLICADOS: Si ya vimos este nombre de archivo, lo saltamos
            if filename in processed_filenames:
                continue

            try:
                # Formato RAVDESS: 03-01-06-01-01-01-01.wav (Emoción es el 3er número)
                parts = filename.split('-')

                if len(parts) >= 3 and parts[2] in emotion_map:

                    # Marcamos como procesado
                    processed_filenames.add(filename)

                    emotion_code = parts[2]
                    emotion_label = emotion_map[emotion_code]
                    file_path = os.path.join(dirname, filename)

                    # Cargar y extraer (usando sr=16000 para consistencia)
                    audio_data, _ = librosa.load(file_path, sr=SAMPLE_RATE)
                    features = extract_features(audio_data, SAMPLE_RATE)
                    features['emotion'] = emotion_label

                    processed_data.append(features)

            except Exception as e:
                print(f"Error en {filename}: {e}")
                continue

# --- 6. GUARDAR CSV ---
if processed_data:
    df = pd.DataFrame(processed_data)

    # Ordenar columnas
    cols = ['emotion'] + [col for col in df.columns if col != 'emotion']
    df = df[cols]

    output_filename = 'ravdess_test_colab.csv'
    df.to_csv(output_filename, index=False)

    print(f"\n¡Proceso completado!")
    print(f"Archivos únicos procesados: {len(df)}")
    print(f"Archivo guardado como: {output_filename}")

    # Código para descargar el archivo automáticamente a tu PC
    from google.colab import files
    files.download(output_filename)
else:
    print("No se encontraron datos para procesar.")

Descargando RAVDESS con kagglehub...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
Dataset descargado en: /kaggle/input/ravdess-emotional-speech-audio

Iniciando procesamiento de audios...

¡Proceso completado!
Archivos únicos procesados: 1440
Archivo guardado como: ravdess_test_colab.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Dataset nuestros audios

In [ ]:
import librosa
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# --- CONFIGURACIÓN ---
local_folder = "/content/drive/MyDrive/Practica_3_PH_Emociones/mis_audios"      # Nombre de tu carpeta
SAMPLING_RATE = 16000            # Frecuencia de muestreo estándar
output_filename = "mis_audios_metrics" # Nombre del archivo de salida

def extract_features(audio_data, sample_rate, n_mfcc=40):
    """
    Misma función de extracción para mantener compatibilidad.
    """
    features = {}

    # Asegurarse de que el audio sea flotante
    if audio_data.dtype != np.float32:
        audio_data = audio_data.astype(np.float32)

    # MFCCs
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    for i, mfcc_val in enumerate(mfccs_mean):
        features[f'mfcc_{i+1}_mean'] = mfcc_val

    # Chroma
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    features['chroma_mean'] = np.mean(chroma)

    # Mel Spectrogram
    mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    features['mel_mean'] = np.mean(mel)

    # Contraste Espectral
    contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sample_rate)
    features['contrast_mean'] = np.mean(contrast)

    return features

# --- SCRIPT PRINCIPAL ---

processed_data = []

if os.path.exists(local_folder):
    print(f"--- Buscando audios en la carpeta '{local_folder}' ---")
    local_files = [f for f in os.listdir(local_folder) if f.lower().endswith(('.wav', '.mp3', '.m4a', '.flac', '.ogg'))]

    if local_files:
        print(f"Se encontraron {len(local_files)} archivos. Procesando...")

        for filename in tqdm(local_files):
            file_path = os.path.join(local_folder, filename)
            try:
                # Cargar audio
                audio_data, sample_rate = librosa.load(file_path, sr=SAMPLING_RATE)

                # Extraer características
                features = extract_features(audio_data, sample_rate)

                # Etiqueta: Usamos '?' para que Orange sepa que debe predecirla
                # O puedes poner el nombre del archivo para identificarlo luego
                features['emotion'] = '?'
                features['filename'] = filename # Añadimos el nombre para que sepas cuál es cual

                processed_data.append(features)
            except Exception as e:
                print(f"Error en {filename}: {e}")
    else:
        print(f"La carpeta '{local_folder}' está vacía.")
else:
    print(f"ERROR: No existe la carpeta '{local_folder}'. Créala y sube tus audios.")

# --- GUARDAR RESULTADOS ---
if processed_data:
    print("\nGenerando archivos Excel y CSV...")
    df = pd.DataFrame(processed_data)

    # Ordenar columnas: poner filename y emotion primero
    cols = ['filename', 'emotion'] + [col for col in df.columns if col not in ['filename', 'emotion']]
    df = df[cols]

    # Guardar Excel
    df.to_excel(f"{output_filename}.xlsx", index=False)
    print(f"¡Listo! Archivo guardado como: {output_filename}.xlsx")

    # Previsualización
    print(df.head())
else:
    print("No se generó ningún archivo porque no hubo datos.")

--- Buscando audios en la carpeta '/content/drive/MyDrive/Practica_3_PH_Emociones/mis_audios' ---
Se encontraron 4 archivos. Procesando...


100%|██████████| 4/4 [00:01<00:00,  2.33it/s]



Generando archivos Excel y CSV...
¡Listo! Archivo guardado como: mis_audios_metrics.xlsx
              filename emotion  mfcc_1_mean  mfcc_2_mean  mfcc_3_mean  \
0  Frase_Felicidad.wav       ?  -391.139252   103.671509   -12.667977   
1        Frase_Ira.wav       ?  -417.494873    89.615089   -15.923532   
2    Frase_Neutral.wav       ?  -449.398254    80.846245    -5.330638   
3   Frase_Tristeza.wav       ?  -443.144775   125.426842    -7.191640   

   mfcc_4_mean  mfcc_5_mean  mfcc_6_mean  mfcc_7_mean  mfcc_8_mean  ...  \
0    41.431877    -1.082888    -1.081947    -4.123047    -4.281711  ...   
1    47.248859    -3.212719     1.737698     0.765870    -3.559467  ...   
2    32.807568    -0.146659     6.319160     1.672086    -7.004560  ...   
3    49.907520     3.756506     6.658509     0.877906    -3.039200  ...   

   mfcc_34_mean  mfcc_35_mean  mfcc_36_mean  mfcc_37_mean  mfcc_38_mean  \
0      4.206089      6.586481      6.938267      8.521297      8.401066   
1     -0.333785   